In [21]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
# import matplotlib.pyplot as plt

In [2]:
data = load_breast_cancer()

In [3]:
X = data.data
y = data.target

In [69]:
class CancerDataset(Dataset):
    
    def __init__(self, X:np.array, y:np.array, transform=None):
        """
        Args:
            X: Features.
            y: target.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.X = X.astype('float32')
        self.y = y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        self.n_samples = self.X.shape[0]
        self.transform = transform

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        
        sample = self.X[idx], self.y[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample
    


In [70]:
input_size = 30
hl_1 = 100
num_classes = 1

batch_size = 200


# This will apply the softmax
train_dataset = CancerDataset(X,y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

In [71]:
X.shape

(569, 30)

In [72]:
# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size, hl_1, num_classes).to(device)

In [73]:
1

1

In [74]:
train_model(train_loader, model, 10, device)

epoch 1 / 10, step 1/3, loss = 2.3851
epoch 1 / 10, step 2/3, loss = 62.0000
epoch 1 / 10, step 3/3, loss = 65.0888
epoch 2 / 10, step 1/3, loss = 67.5000
epoch 2 / 10, step 2/3, loss = 69.0000
epoch 2 / 10, step 3/3, loss = 49.7041
epoch 3 / 10, step 1/3, loss = 65.5000
epoch 3 / 10, step 2/3, loss = 62.0000
epoch 3 / 10, step 3/3, loss = 60.3550
epoch 4 / 10, step 1/3, loss = 61.0000
epoch 4 / 10, step 2/3, loss = 64.0000
epoch 4 / 10, step 3/3, loss = 63.3136
epoch 5 / 10, step 1/3, loss = 61.0000
epoch 5 / 10, step 2/3, loss = 60.5000
epoch 5 / 10, step 3/3, loss = 67.4556
epoch 6 / 10, step 1/3, loss = 64.0000
epoch 6 / 10, step 2/3, loss = 57.5000
epoch 6 / 10, step 3/3, loss = 67.4556
epoch 7 / 10, step 1/3, loss = 63.5000
epoch 7 / 10, step 2/3, loss = 62.5000
epoch 7 / 10, step 3/3, loss = 62.1302
epoch 8 / 10, step 1/3, loss = 63.0000
epoch 8 / 10, step 2/3, loss = 65.5000
epoch 8 / 10, step 3/3, loss = 59.1716
epoch 9 / 10, step 1/3, loss = 63.0000
epoch 9 / 10, step 2/3, lo

In [75]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        inputs = inputs.to(device)
        targets = targets.to(device)
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.cpu().detach().numpy()
        actual = targets.cpu().numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc



evaluate_model(train_loader, model)

0.37258347978910367